In [1]:
from qwertyenv.collect_coins_pz import CollectCoinsEnv
from qwertyenv.pz_to_gymnasium_wrapper import PZ2GymnasiumWrapper
from qwertyenv.ensure_valid_action_pz import EnsureValidAction
# import gym
import qwertyenv
import numpy as np

import torch

import tianshou
from tianshou.data import Collector
from tianshou.env import DummyVectorEnv

# from stable_baselines3 import PPO
# from stable_baselines3.ppo.policies import MlpPolicy, MultiInputPolicy
# from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
# import pdb

In [3]:
action = None

# env =  gym.make('qwertyenv/CollectCoins-v0', pieces=['rock', 'rock'])

pz_env = CollectCoinsEnv(pieces=['rock', 'rock'], with_mask=True)

def another_action_taken(action_taken):
    global action
    action = action_taken

# Wrapping the original environment as to make sure a valid action will be taken.
pz_env = EnsureValidAction(
  pz_env,
  pz_env.check_action_valid,
  pz_env.provide_alternative_valid_action,
  another_action_taken
)

def act_player_1(obs):
    return (0, 0)

gym_env = PZ2GymnasiumWrapper(pz_env, act_others={'player_1': act_player_1})

policy = tianshou.policy.RandomPolicy()
# agent_w = PPO(MultiInputPolicy, env)

In [4]:
def play(num_episodes: int = 1):

  seed = 1
    
  # ======== environment setup =========
  train_envs = DummyVectorEnv([lambda: gym_env])
  test_envs = DummyVectorEnv([lambda: gym_env])
  # seed
  np.random.seed(seed)
  torch.manual_seed(seed)
  train_envs.seed(seed)
  test_envs.seed(seed)
    
  policy.eval()
  # pdb.set_trace()
  test_collector = Collector(policy, test_envs, exploration_noise=False)



  result = test_collector.collect(n_step=10000, render=False) # n_episode=num_episodes,
    
  print(result['rew'])
    
    # test_collector = Collector(policy, self.test_envs, exploration_noise=False)
    # for episode in range(num_episodes):
    #     print(f'{episode=}')
    #     print("-------")
    #     obs, info = env.reset(seed=1)
    #     obs = obs[()]
    #     env.render()
    #     while True:
    #       action, _state = policy.forward({'obs': obs}) #  agent_w.predict(obs, deterministic=True)
    #       # print(f'action predicted: {action}')
    #       obs, reward, done, info = env.step(action)
    #       # print(f'action taken: {action}')
    #       env.render()
    #       print()
    #       print(f'{reward=}, {done=}, {info=}')
    #       print()
    #       if done:
    #         break
    #     print()

In [5]:
play(1)

0.2274000000000001


In [6]:
# TODO: agent_w.learn(total_timesteps=int(2e5), progress_bar=True)

In [7]:
play(1)

0.18340000000000006


In [8]:
# TODO: mean_reward, std_reward = evaluate_policy(agent_w, env, deterministic=True)
# print(f'{mean_reward=}, {std_reward=}')